In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(1, '/home/cem/Documents/imps/src')

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree as KDTree
import trimesh
import igl
import torch
from tqdm import tqdm
from matplotlib import cm
from imps.data import ScanNetScene

from torch.nn import functional as F
from imps.ndf.local_model import MiniNDF, NDF
from imps.sqn.model import SQN
from imps.sqn.data_utils import prepare_input

DATA_PATH = '/home/cem/Documents/datasets/ScanNet/scans/'
SCENE_NAME = 'scene0000_00'

RES = 75
SIGMAS = [0.09, 0.03, 0.01] # noise for grid points
N_POINTS = int(2e4)

In [ ]:
vis = scene.create_if_data(
    RES, N_POINTS, SIGMAS, o3d_format=True, scale_sigmas=False
)

In [ ]:
o3d.visualization.draw_geometries([vis[3]])

In [ ]:
scene = ScanNetScene(os.path.join(DATA_PATH, SCENE_NAME))
voxels, surface_points, surface_colors, vicinities, distances, _ = scene.create_if_data(
    RES, N_POINTS, SIGMAS, o3d_format=False, scale_sigmas=False
)

In [ ]:
model = NDF()
# checkpoint = torch.load('../../data/ndf_125_model')
# model.load_state_dict(checkpoint['model_state_dict'])
# model = SQN(d_feature=6, d_in=16, encoder_dims=[32, 64, 128], decoder_dims=[64, 32, 16, 1],
#          activation=torch.nn.ReLU(inplace=True))
model.to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# features = torch.FloatTensor(np.concatenate([surface_points, surface_colors], axis=-1)).unsqueeze(0).to('cuda')
# xyz = torch.FloatTensor(surface_points).unsqueeze(0)
# query = torch.FloatTensor(vicinities).unsqueeze(0).to('cuda')
# query_distances = torch.FloatTensor(distances).unsqueeze(0).to('cuda')

# input_points, input_neighbors, input_pools = prepare_input(xyz, k=16, num_layers=4, sub_sampling_ratio=4, 
#                                                            device='cuda')

query_points = torch.FloatTensor(vicinities).unsqueeze(0).cuda()
query_distances = torch.FloatTensor(distances).unsqueeze(0).cuda()
voxel_grid = torch.FloatTensor(voxels).unsqueeze(0).cuda()

In [ ]:
model.train()

losses = []
for e in tqdm(range(1000)):
    optimizer.zero_grad()
    pred_dist = model.forward(query_points, voxel_grid)
    # pred_dist = model.forward(features, input_points, input_neighbors, input_pools, query)
    pred_dist = pred_dist.squeeze(-1)

    dist_criterion = torch.nn.L1Loss(reduction='none')    
    loss = dist_criterion(torch.clamp(pred_dist, max=0.25), 
                               torch.clamp(query_distances, max=0.25)).sum(dim=-1).mean()

    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if e%100 == 0:
        print(losses[-1])
        torch.save({'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict()}, 
                   '../../data/ndf_75_model')

In [ ]:
batch['grid_coords'] = batch['grid_coords'].to('cpu')
batch['df'] = batch['df'].to('cpu')
batch['inputs'] = batch['inputs'].to('cpu')

In [ ]:
model.eval()

with torch.no_grad():
    pred_dist = model.forward(query_points, voxel_grid)
    pred_dist = pred_dist.squeeze(-1)
    
# pred_dist = pred_dist.detach().cpu().numpy().squeeze()
pred_dist = distances

viridis = cm.get_cmap('Reds')
dist_norm = (pred_dist-pred_dist.min()) / (pred_dist.max()-pred_dist.min())

vicinity_pcd = o3d.geometry.PointCloud()
vicinity_pcd.points = o3d.utility.Vector3dVector(vicinities)
vicinity_pcd.colors = o3d.utility.Vector3dVector(viridis(pred_dist)[..., :-1])

o3d.visualization.draw_geometries([vicinity_pcd])

In [ ]:
import time

model.eval()

start = time.time()
num_steps = 5
num_points = 100000
filter_val = 0.009
threshold = 0.1

for param in model.parameters():
    param.requires_grad = False

sample_num = 40000
samples_cpu = np.zeros((0, 3))

#sample_idxs = np.random.choice(boundary_points.shape[0], sample_num, replace=False)  
#samples = torch.tensor(boundary_points[None, sample_idxs, ...], dtype=torch.float).to('cuda')
r1, r2 = scene.bounds
r1 = np.copy(r1)
samples = (torch.rand(sample_num, 3)*(r1 - r2)+r2).float().to('cuda').reshape(1, sample_num, 3)
# samples = torch.Tensor(grid_points.reshape(1, -1, 3)).float().to('cuda')

# idxs = np.random.choice(grid_points.shape[0], sample_num, replace=False)  
# samples = grid_points[idxs]
# samples[:, -1] = ()

# samples_pcd = o3d.geometry.PointCloud()
# samples_pcd.points = o3d.utility.Vector3dVector(samples)

# samples = torch.Tensor(samples.reshape(1, len(idxs), 3)).float().to('cuda')

encoding = model.encoder(voxel_grid)
#input_points, input_neighbors, input_pools = prepare_input(xyz, k=16, num_layers=3, sub_sampling_ratio=4, 
#                                                           device='cuda')

# encoding = model.encoder(features, input_points, input_neighbors, input_pools)
samples.requires_grad = True

i = 0
while len(samples_cpu) < num_points:
    print('iteration', i)

    for j in range(num_steps):
        # print(j)
        df_pred = torch.clamp(model.decoder(samples, *encoding), max=threshold)
        
        # df_pred = torch.clamp(model.decoder(samples, encoding, input_points), max=threshold).squeeze(-1)
        df_pred.sum().backward()

        gradient = samples.grad.detach()
        samples = samples.detach()
        df_pred = df_pred.detach()
        voxel_grid = voxel_grid.detach()
        samples = samples - (F.normalize(gradient, dim=2) * df_pred.unsqueeze(-1))
        samples = samples.detach()
        samples.requires_grad = True


    if not i == 0:
        samples_cpu = np.vstack((samples_cpu, samples[df_pred < filter_val].detach().cpu().numpy()))

    samples = samples[df_pred < 0.03].unsqueeze(0)
    indices = torch.randint(samples.shape[1], (1, sample_num))
    samples = samples[[[0, ] * sample_num], indices]
    samples += (threshold / 3) * torch.randn(samples.shape).to('cuda')  # 3 sigma rule
    samples = samples.detach()
    samples.requires_grad = True

    i += 1
    print("\t", samples_cpu.shape)

duration = time.time() - start

In [ ]:
rec_pcd = o3d.geometry.PointCloud()
rec_pcd.points = o3d.utility.Vector3dVector(samples_cpu)
# rec_pcd.estimate_normals()
o3d.visualization.draw_geometries([rec_pcd]) #voxel_grid])

In [ ]:
rec_pcd.estimate_normals()
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(rec_pcd, depth=9)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
radii = [0.05, 0.1, 0.2, 0.4]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    rec_pcd, o3d.utility.DoubleVector(radii))

In [ ]:
rec_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([rec_mesh])

In [ ]:
def get_cube(cube_center, cube_size, n=10):
    x_min, x_max = cube_center[0]-cube_size/2, cube_center[0]+cube_size/2
    y_min, y_max = cube_center[1]-cube_size/2, cube_center[1]+cube_size/2
    z_min, z_max = cube_center[2]-cube_size/2, cube_center[2]+cube_size/2

    x_ = np.linspace(x_min, x_max, n)
    y_ = np.linspace(y_min, y_max, n)
    z_ = np.linspace(z_min, z_max, n)
    
    X, Y, Z = np.meshgrid(x_, y_, z_, indexing='ij')
    return np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T

def surface_grad(model, input, initial_points, num_steps, step_size):
    for param in model.parameters():
        param.requires_grad = False
            
    encoding = model.encoder(input)

    samples = torch.Tensor(initial_points.reshape(1, len(initial_points), 3)).float().to('cuda')
    samples.requires_grad = True

    sample_steps = np.zeros((num_steps, len(initial_points), 3))

    i = 0

    for j in range(num_steps):
        # Update this in NDF mocel
        df_pred = model.decoder(samples, *encoding)
        df_pred.sum().backward()
        
        gradient = samples.grad.detach()
        samples = samples.detach()
        df_pred = df_pred.detach()
        input = input.detach()
        
        samples = samples - step_size*(F.normalize(gradient, dim=2)*df_pred.reshape(-1, 1))

        samples = samples.detach()
        samples_cpu = samples.detach().cpu().numpy()[0]
        sample_steps[j] = samples_cpu
        
        samples.requires_grad = True
        
    return sample_steps

In [ ]:
# TODO: Add initial points as well
r1, r2 = scene.bounds
cube_size = 0.1
cube_x_ = np.arange(r1[0]+cube_size/2, r2[0]-cube_size/2, cube_size)
cube_y_ = np.arange(r1[1]+cube_size/2, r2[1]-cube_size/2, cube_size)
cube_z_ = np.arange(r1[2]+cube_size/2, r2[2]-cube_size/2, cube_size)
cube_X, cube_Y, cube_Z = np.meshgrid(cube_x_, cube_y_, cube_z_, indexing='ij')

cube_centers = np.vstack([cube_X.ravel(), cube_Y.ravel(), cube_Z.ravel()]).T 
all_steps = []
init_steps = []

for cc in tqdm(cube_centers):
    init_points = get_cube(cc, cube_size)
    init_steps.append(init_points)
    surface_steps = surface_grad(model, voxel_grid, init_points, 5, 0.5)
    all_steps.append(surface_steps)

init_steps = np.concatenate(init_steps, axis=0)[None, :, :]
all_steps = np.concatenate(all_steps, axis=1)
all_steps = np.concatenate([init_steps, all_steps], axis=0)

In [ ]:
import pickle
pickle.dump(all_steps, open('../../data/ndf_steps.pkl', 'wb'))